In [9]:
N = 9
c = 4
order = 6
k = 5

In [10]:
import netgen.meshing as ngm
from netgen.geom2d import unit_square
from ngsolve import *

ngmesh = ngm.Mesh()
ngmesh.SetGeometry(unit_square)
ngmesh.dim = 2
pnums = []
for i in range(N + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(ngm.MeshPoint(ngm.Pnt(i / N, j / N, 0))))

foo = ngm.FaceDescriptor(surfnr=1,domin=1,bc=1)
ngmesh.Add (foo)
ngmesh.SetMaterial(1, "mat")
for j in range(N):
    for i in range(N):
        ngmesh.Add(ngm.Element2D(1, [pnums[i + j * (N + 1)],
                                 pnums[i + (j + 1) * (N + 1)],
                                 pnums[i + 1 + (j + 1) * (N + 1)],
                                 pnums[i + 1 + j * (N + 1)]]))
for i in range(N):
   ngmesh.Add(ngm.Element1D([pnums[N + i * (N + 1)], pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(ngm.Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))
for i in range(N):
   ngmesh.Add(ngm.Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(ngm.Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))

mesh = Mesh(ngmesh)
# Draw(mesh)
# print("boundaries" + str(mesh.GetBoundaries()))

In [11]:
from ngsolve import *
from trefftzngs import *
import numpy as np
# fes = L2(mesh, order=order, flags = { "dgjumps" : True })

truesol =  sin( k*(c*x + y) )#exp(-pow(c*x+y,2)))#
v0 = c*k*cos(k*(c*x+y))#grad(U0)[0]
sig0 = -k*cos(k*(c*x+y))#-grad(U0)[1]
# U0 = GridFunction(fes)
# U0.Set(truesol)
# Draw(U0,mesh,'U0')

In [12]:
def MakeSystem(truesol,v0,sig0,fes):
    U = fes.TrialFunction()
    V = fes.TestFunction()

    v = grad(U)[0]
    sig = -grad(U)[1]
    w = grad(V)[0]
    tau = -grad(V)[1]

    vo = grad(U.Other())[0]
    sigo = -grad(U.Other())[1]
    wo = grad(V.Other())[0]
    tauo = -grad(V.Other())[1]

    h = specialcf.mesh_size
    n = specialcf.normal(2)
    n_t = n[0]/Norm(n)
    n_x = n[1]/Norm(n)

    mean_v = 0.5*(v+vo)
    mean_w = 0.5*(w+wo)
    mean_sig = 0.5*(sig+sigo)
    mean_tau = 0.5*(tau+tauo)

    jump_vx = ( v - vo ) * n_x
    jump_wx = ( w - wo ) * n_x
    jump_sigx = ( sig - sigo ) * n_x
    jump_taux = ( tau - tauo ) * n_x

    jump_vt = ( v - vo ) * n_t
    jump_wt = ( w - wo ) * n_t
    jump_sigt = ( sig - sigo ) * n_t
    jump_taut = ( tau - tauo ) * n_t

    jump_Ut = (U - U.Other()) * n_t
    #gfu.vec.data = a.mat.Inverse() * f.vec

    timelike = n_x**2 #IfPos(n_t,0,IfPos(-n_t,0,1)) # n_t=0
    spacelike = n_t**2 #IfPos(n_x,0,IfPos(-n_x,0,1)) # n_x=0

    alpha = 0.5 #pow(10,5)
    beta = 0.5 #pow(10,5)
    gamma = 1

    a = BilinearForm(fes)
    # a += SymbolicBFI( spacelike * ( IfPos(n_t,v,vo)*(pow(c,-2)*jump_wt+jump_taux) + IfPos(n_t,sig,sigo)*(jump_wx+jump_taut) ) ,VOL,  skeleton=True ) #space like faces
    a += SymbolicBFI( spacelike * ( IfPos(n_t,v,vo)*(pow(c,-2)*jump_wt) + IfPos(n_t,sig,sigo)*(jump_taut) ) ,VOL,  skeleton=True ) #space like faces, no jump in x since horizontal
    a += SymbolicBFI( timelike 	* ( mean_v*jump_taux + mean_sig*jump_wx + alpha*jump_vx*jump_wx + beta*jump_sigx*jump_taux ) ,VOL, skeleton=True ) #time like faces
    a += SymbolicBFI( spacelike * IfPos(n_t,1,0) * ( pow(c,-2)*v*w + sig*tau ), BND, skeleton=True) #t=T (or *x)
    a += SymbolicBFI( timelike 	* ( sig*n_x*w + alpha*v*w ), BND, skeleton=True) #dirichlet boundary 'timelike'
    a += SymbolicBFI( spacelike * ( gamma * (jump_Ut)*IfPos(n_t,V.Other(),V) ) ,VOL,  skeleton=True ) #correction term to recover sol of second order system
    a += SymbolicBFI( spacelike * ( gamma * IfPos(-n_t,1,0) * U*V ) ,BND,  skeleton=True ) #BND correction term to recover sol of second order system
    a.Assemble()

    f = LinearForm(fes)
    f += SymbolicLFI( spacelike * IfPos(-n_t,1,0) *  ( pow(c,-2)*v0*w + sig0*tau ), BND, skeleton=True) #t=0 (or *(1-x))
    f += SymbolicLFI( timelike 	* ( v0 * (alpha*w - tau*n_x) ), BND, skeleton=True) #dirichlet boundary 'timelike'
    f += SymbolicLFI( spacelike * gamma * IfPos(-n_t,1,0) *  ( (truesol)*V ) ,BND,  skeleton=True ) #rhs correction term to recover sol of second order system
    f.Assemble()

    return [a,f]
    # gfu2= GridFunction(fes, name="uDG")
    # gfu2.vec.data = a.mat.Inverse() * f.vec

In [13]:
def SolveNp(a,f,fes):
	gfu = GridFunction(fes, name="uDG")

	nmat = np.zeros((a.mat.height,a.mat.width))
	nvec = np.zeros(a.mat.width)

	for i in range(a.mat.width):#gfu.vec.data = a.mat.Inverse() * f.vec
		for j in range(a.mat.height):
			nmat[j,i] = a.mat[j,i]
	nvec = f.vec.FV().NumPy() #nvec

	sol = np.linalg.solve(nmat,nvec)
	for i in range(a.mat.height):
		gfu.vec[i] = sol[i]
	# print("cond nmat: ", np.linalg.cond(nmat))
	return [gfu,np.linalg.cond(nmat)]

In [ ]:
def error(gfu,truesol,fes):
	U0 = GridFunction(fes)
	U0.Set(truesol)
	L2error = Integrate((truesol - gfu)*(truesol - gfu), mesh)
	sH1error = Integrate((grad(U0) - grad(gfu))*(grad(U0) - grad(gfu)), mesh)
	# print("error=", L2error)
	# print("grad-error=", sH1error)
	#Draw(gfu,mesh,'sol')
	#Draw(grad(gfu),mesh,'gradsol')
	return [L2error,sH1error]

In [ ]:
solution = []
for order in range(3,15):
    for btype in range(0,3):
        print("run order nr.: " + str(order))
        fes = FESpace("trefftzfespace", mesh, order = order, wavespeed = c, dgjumps=True, basistype = btype)
        [a,f] = MakeSystem(truesol,v0,sig0,fes)
        [gfu,cond] = SolveNp(a,f,fes)
        [L2error,sH1error] = error(gfu,truesol,fes)
        solution.append([btype,order,cond,L2error,sH1error])
        print("btype: " + str(btype) + " cond: " + str(cond) + " L2error: " + str(L2error) + " H1error: "+ str(sH1error))

run order nr.: 3
btype: 0 cond: 1321.70275429 L2error: 0.002222801728686229 H1error: 1.7619738690604119
run order nr.: 3
btype: 1 cond: 3612.1722111 L2error: 0.002222801728686588 H1error: 1.7619738690605595
run order nr.: 3
btype: 2 cond: 5264.62597407 L2error: 0.002222801728685463 H1error: 1.7619738690604725
run order nr.: 4
btype: 0 cond: 25198.3194783 L2error: 1.8349500554512512e-05 H1error: 0.02910945464569372
run order nr.: 4
btype: 1 cond: 23948.1861456 L2error: 1.8349500554119685e-05 H1error: 0.029109454645735756
run order nr.: 4
btype: 2 cond: 19369.1966264 L2error: 1.834950055533573e-05 H1error: 0.029109454645483996
run order nr.: 5
btype: 0 cond: 99526.2982819 L2error: 1.114344441125595e-07 H1error: 0.0005127924594675741
run order nr.: 5
btype: 1 cond: 170306.766985 L2error: 1.1143444371451598e-07 H1error: 0.0005127924593529844
run order nr.: 5
btype: 2 cond: 120403.364604 L2error: 1.1143444420328575e-07 H1error: 0.000512792459413399
run order nr.: 6
btype: 0 cond: 2920681.91

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

data = ['condition', 'L2error', 'H1error']
sol = np.array(solution)

for i in range(len(data)):
    fig = plt.figure()
    ax = fig.gca()
    plt.semilogy(sol[sol[:,0]==0,1], sol[sol[:,0]==0,i+2], '-o', label="Monomials")
    plt.semilogy(sol[sol[:,0]==1,1], sol[sol[:,0]==1,i+2], '--x', label="Legendre")
    plt.semilogy(sol[sol[:,0]==2,1], sol[sol[:,0]==2,i+2], '--s', label="Chebyshev")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.title('Wavespeed: ' + str(c) + " Meshsize: " + str(1/N))
    plt.legend()
    plt.ylabel(data[i])
    plt.xlabel('order')
    plt.savefig("results/"+data[i]+".png")
    plt.show()